In [52]:
import torch, torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
from IPython.display import Image
from torchvision.utils import save_image
import random
import pandas as pd
import numpy as np

In [57]:
#Helpers
def show_data(X):
    plt.figure(figsize=(10, 10))
    i = 1
    for img in X:
        print(".", end="")
        plt.subplot(10, 10, i)
        plt.imshow(img.reshape((28, 28)), cmap="gray")
        i+=1
        if i>100: 
            break
    plt.show()

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [10]:
data = pd.read_csv("../input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv").astype('float32')

In [ ]:
indexes = {
    "a": (0, 13869), #13869 values
    "b": (13869, 22538), #8668
    "c": (22538, 45947), #23409
    "d": (45947, 56081), #10134
    "e": (56081, 67521), #11440
    "f": (67521, 68684), #1163,
    "g": (68684, 74446), #5762
    "h": (74446, 81664), #7218
    "i": (81664, 82784), #1120,
    "j": (82784, 91277), #8493,
    "k": (91277, 96880), #5603,
    "l": (96880, 108466), #11586,
    "m": (108466, 120802), #12336,
    "n": (120802, 139812), #19010,
    "o": (139812, 197637), #57825,
    "p": (197637, 216978), #19341,
    "q": (216978, 222790), #5812,
    "r": (222790, 234356), #11566,
    "s": (234356, 282775), #48419,
    "t": (282775, 305270), #22495,
    "u": (305270, 334278), #29008,
    "v": (334278, 338460), #4182,
    "w": (338460, 349244), #10784,
    "x": (349244, 355516), #6272,
    "y": (355516, 366375), #10859,
    "z": (366375, 372451), #6076,
}

abba = pd.concat((data.iloc[indexes[letter][0]:indexes[letter][1]].sample(frac = 1).iloc[:4182].iloc[:,1:] for letter in indexes), ignore_index=True)

In [54]:
X = abba.values
X = X.reshape((abba.shape[0], 28, 28))

In [56]:
# Normalization
X = (X - 127.5) / 127.5
show_data(X) #Shows first hundred

In [ ]:
X = torch.tensor(X)
# Showing Random Image
img = X[random.randint(0, X.shape[0])]
plt.imshow(img.squeeze(0), cmap="gray")

In [12]:
bs = 100
data_loader = DataLoader(X, bs, shuffle=True)
show_data(next(iter(data_loader)))
next(iter(data_loader)).shape

In [14]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
device

In [15]:
from torch.nn.modules.linear import Linear
slope = 0.2 #for leakyReLU
Dnet = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(slope),
    nn.Linear(256, 256),
    nn.LeakyReLU(slope),
    nn.Linear(256, 1),
    nn.Sigmoid()
)

In [16]:
Dnet.to(device)

In [17]:
latent_size=64
Gnet = nn.Sequential(
    nn.Linear(latent_size, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 784),
    nn.Tanh() #This because we want -1 to 1 values
)

In [18]:
Gnet.to(device)

In [19]:
#Loss Function
criterion = nn.BCELoss() 
#Optimizers
dnet_optimizer = torch.optim.Adam(Dnet.parameters(), lr=0.0002)
gnet_optimizer = torch.optim.Adam(Gnet.parameters(), lr=0.0002)
#Reset Grad
def reset_grad(): # After every gradient dcent iteration, we need toreset the gradients
  dnet_optimizer.zero_grad()
  gnet_optimizer.zero_grad()

In [20]:

def train_discriminator(images):
    # Calculate Real Loss 
    real_labels = torch.ones(bs, 1).to(device)
    fake_labels = torch.zeros(bs, 1).to(device)
    output = Dnet(images)
    d_loss_real = criterion(output, real_labels)
    real_score = output
    # Calculate Fake Loss
    fake_images = Gnet(torch.randn(bs, latent_size).to(device))
    output = Dnet(fake_images)
    d_loss_fake = criterion(output, fake_labels)
    fake_score = output
    # Calculate Total Loss
    d_loss = d_loss_real + d_loss_fake
    
    # Gradient Decent
    d_loss.backward()
    dnet_optimizer.step()
    reset_grad()
    return d_loss, real_score, fake_score

In [21]:
def train_generator():
    # Feedforward and Calculate Loss
    fake_images = Gnet(torch.randn(bs, latent_size).to(device))
    labels = torch.ones(bs, 1).to(device)
    descriminator_thinks = Dnet(fake_images)
    g_loss = criterion(descriminator_thinks, labels)
  
    # Gradient Decent
    g_loss.backward()
    gnet_optimizer.step()
    reset_grad()
    return g_loss, fake_images



In [22]:
# Creating Folder to Save Stuff 
import os
folder = 'samples'
if not os.path.exists(folder):
    os.makedirs(folder)

In [23]:
# Save real images
for images in data_loader:
    images = images.reshape(images.size(0), 1, 28, 28)
    save_image(denorm(images), os.path.join(folder, 'real_images.png'), nrow=10)
    break
    
Image(os.path.join(folder, 'real_images.png'))

In [24]:
# Saving Fkae Images Before Training
sample_vectors = torch.randn(bs, latent_size).to(device)

def save_fake_images(index):
    fake_images = Gnet(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'fake_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(denorm(fake_images), os.path.join(folder, fake_fname), nrow=8)
    Image('./samples/{}'.format(fake_fname))
    return fake_fname

Image(os.path.join(folder, save_fake_images(0)))

In [25]:
epochs = 450
total_step = len(data_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(epochs):
    for i, images in enumerate(data_loader):
        if images.shape[0] == bs:
            images_vector = images.view(bs, -1).to(device)
        else:
            continue
        d_loss, real_score, fake_score = train_discriminator(images_vector)
        g_loss, fake_images = train_generator()
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'.format(epoch, epochs, i+1, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))
    save_fake_images(epoch+1)


In [48]:
# Graphing Losses
plt.figure(figsize=(20, 10))
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('step')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [61]:
# Seeing it go
sample_vectors = torch.randn(bs, latent_size).to(device)
trial_index = 7892740524752093723249203
Image(os.path.join(folder, save_fake_images(trial_index)))